In [ ]:
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
from uuid import uuid4
from general_fun import *

In [ ]:
engine, conn = connect_database()

In [ ]:
# Creating a dataframe with address of locations we want to reterive
locat = ["Genieweg 38, 1566NM Assendelft","Macuarima 131a, Santa Cruz, Aruba", "Lindberghlaan 122, 2497EC Den Haag","Merelplein 4, 2771SB Boskoop"]
df = pd.DataFrame({'add': locat})
 
# Creating an instance of Nominatim Class
geolocator = Nominatim(user_agent="my_request")
 
# applying the rate limiter wrapper
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
 
# Applying the method to pandas DataFrame
df['location'] = df['add'].apply(geocode)
df['Lat'] = df['location'].apply(lambda x: x.latitude if x else None)
df['Lon'] = df['location'].apply(lambda x: x.longitude if x else None)

In [ ]:
geometry = gpd.points_from_xy(df["Lon"], df["Lat"])
geo_df = gpd.GeoDataFrame(df[["add", "location", "Lat", "Lon"]], geometry=geometry)
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]
geo_df.head()

In [ ]:
locations = []
for coordinates in geo_df_list:
    address = geolocator.reverse(f"{coordinates[0]}, {coordinates[1]}").raw['address']
    address["latitude"] = coordinates[0]
    address["longitude"] = coordinates[1]
    locations.append(address)

In [ ]:
df = pd.DataFrame(locations)

df["street"] = df["road"]+ " " + df["house_number"]
df['location'] = df[['village', 'town', 'city']].stack().groupby(level=0).agg('_'.join)
df['ISO3166-2'] = df[['ISO3166-2-lvl4', 'ISO3166-2-lvl3']].stack().groupby(level=0).agg('_'.join)
df["id"] = df.index + 1

In [ ]:
house = pd.DataFrame(data=None, columns=["id", "location", "street", "postcode", 'municipality', "state", "country", "ISO3166-2", "latitude", "longitude"])

for col in house.columns:
    house[col] = df[col]

house["date_from"] = ["1985-02-17", "2007-02-03", "2009-03-01", "2017-04-19"]
house["date_until"] = ["2009-03-01", "2007-08-22", "2017-07-01", None]

house["date_from"] = pd.to_datetime(house["date_from"], format='%Y-%m-%d', utc=False)
house["date_until"] = pd.to_datetime(house["date_until"], format='%Y-%m-%d', utc=False)

In [ ]:
house.to_sql('house', conn, index=False, if_exists='replace', index_label="id")
house.to_pickle("house.pkl")

In [ ]:
Map = folium.Map(location=np.array(house.loc[house["location"].isin(["Boskoop"]), ["latitude", "longitude"]])[0], tiles="OpenStreetMap", zoom_start=15)

for jj, oordinates in enumerate(geo_df_list):
    Map.add_child(folium.Marker(location=coordinates, popup=locations[jj], icon=folium.Icon(color='red', icon='home')))

Map